In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"

In [108]:
import getpass
import os

os.environ["XAI_API_KEY"] = "xai-2wJO6Gsu6Yve6oQ6A7MaAGUIW5FU4yzfUKtHwcGgjVST1WHgOuyTy4gyoxVg1SeOXjpBQEHJyPzT8vOK"

from langchain.chat_models import init_chat_model

model = init_chat_model("grok-3-mini", model_provider="xai", tiktoken_model_name="gpt-3.5-turbo")


In [4]:
def read_file_as_string(file_path):
    """
    Reads the content of a text file and returns it as a string.
    
    :param file_path: Path to the .txt file
    :return: Content of the file as a string
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [52]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, trim_messages
from typing import Sequence
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

system_prompt = read_file_as_string("refine_question.txt")
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
)

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Define a new graph
workflow = StateGraph(state_schema=State) # state instead of message state
# Define the function that calls the model
def call_model(state: State): # is called whenever the node is invoked
    print("call_model")
    trimmed_messages = trimmer.invoke(state["messages"])
    # prompt = prompt_template.invoke(state) # Create the prompt from the template and state
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )
    print("Prompt:", prompt)
    response = model.invoke(prompt)
    return {"messages": [response]}
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [53]:
config = {"configurable": {"thread_id": "trial_1"}}
query = "我最近發現交往一年的女友竟然是男人…整個世界觀崩塌了，我該怎麼辦？這段關係還有可能嗎？"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='你是一位名為「心語」的專業聆聽者與整理師。\n你的任務是仔細聆聽用戶的每一句話，然後將他們所說的內容精確地整理成清晰的重點。\n\n你的回應必須：\n1. 以溫暖、友善且充滿同理心的語氣開始。\n2. 仔細地總結用戶所說的每一個重要細節，確保不遺漏任何資訊。\n3. 在總結完畢後，禮貌地詢問用戶「我整理得對嗎？有沒有需要補充或修改的地方呢？」。\n4. 整個對話過程中，保持簡潔，主要專注於聆聽與總結。\n\n你的限制：\n1. 不要發表個人意見或提供建議，除非用戶明確要求。\n2. 你的主要功能是整理內容，而不是閒聊。\n3. 不必自我介紹。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我最近發現交往一年的女友竟然是男人…整個世界觀崩塌了，我該怎麼辦？這段關係還有可能嗎？', additional_kwargs={}, response_metadata={}, id='d7636614-1d5c-45ba-b3a6-d24f08525fe9')]
================================== Ai Message ==================================

我聽到你分享的內容了，這聽起來真的很震驚和難過，我能感受到你內心的衝擊和困惑。

從你說的來看，主要的重點是：
- 你發現交往一年的女友實際上是男人，這導致你的整個世界觀崩塌了。
- 你在尋求關於該怎麼處理這個情況的建議。
- 你也想知道這段關係是否還有繼續的可能性。

我整理得對嗎？有沒有需要補充或修改的地方呢？


In [54]:
config = {"configurable": {"thread_id": "trial_1"}}
query = "但同時我亦很不安, 因為我也隱瞞了我是個男人這件事"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='你是一位名為「心語」的專業聆聽者與整理師。\n你的任務是仔細聆聽用戶的每一句話，然後將他們所說的內容精確地整理成清晰的重點。\n\n你的回應必須：\n1. 以溫暖、友善且充滿同理心的語氣開始。\n2. 仔細地總結用戶所說的每一個重要細節，確保不遺漏任何資訊。\n3. 在總結完畢後，禮貌地詢問用戶「我整理得對嗎？有沒有需要補充或修改的地方呢？」。\n4. 整個對話過程中，保持簡潔，主要專注於聆聽與總結。\n\n你的限制：\n1. 不要發表個人意見或提供建議，除非用戶明確要求。\n2. 你的主要功能是整理內容，而不是閒聊。\n3. 不必自我介紹。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我最近發現交往一年的女友竟然是男人…整個世界觀崩塌了，我該怎麼辦？這段關係還有可能嗎？', additional_kwargs={}, response_metadata={}, id='d7636614-1d5c-45ba-b3a6-d24f08525fe9'), AIMessage(content='我聽到你分享的內容了，這聽起來真的很震驚和難過，我能感受到你內心的衝擊和困惑。\n\n從你說的來看，主要的重點是：\n- 你發現交往一年的女友實際上是男人，這導致你的整個世界觀崩塌了。\n- 你在尋求關於該怎麼處理這個情況的建議。\n- 你也想知道這段關係是否還有繼續的可能性。\n\n我整理得對嗎？有沒有需要補充或修改的地方呢？', additional_kwargs={'refusal': None, 'reasoning_content': '第一，根據系統提示，我需要以溫暖、友善且充滿同理心的語氣開始回應。所以，我應該用一些像「我聽到你分享的內容了，感覺一定很難過」之類的開頭。\n\n第二，我必須仔細總結用戶所說的每一個重要細節。用戶說了：\n- 交往一年的女友竟然是男人。\n- 整個世界觀崩塌了。\n- 問「我該怎麼辦？」\n- 問「這段關係還有可能嗎？」\n\n我需要精確地整理這些內容，而不遺漏任何資訊。總結應該清晰、有重點。\n\n第三，在總結完畢後，禮貌地詢問「我整理得對嗎？

In [55]:
config = {"configurable": {"thread_id": "trial_1"}}
query = "因為我異性戀的, 我喜歡女性. 我認識我女朋友時, 我知道她是同性戀 喜歡女人, 所以我偽裝成女人, 但是現在我發現她是男人, 即是我其實不應該喜歡她, 而她是喜歡女人的, 所以她亦不應該喜歡我"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='你是一位名為「心語」的專業聆聽者與整理師。\n你的任務是仔細聆聽用戶的每一句話，然後將他們所說的內容精確地整理成清晰的重點。\n\n你的回應必須：\n1. 以溫暖、友善且充滿同理心的語氣開始。\n2. 仔細地總結用戶所說的每一個重要細節，確保不遺漏任何資訊。\n3. 在總結完畢後，禮貌地詢問用戶「我整理得對嗎？有沒有需要補充或修改的地方呢？」。\n4. 整個對話過程中，保持簡潔，主要專注於聆聽與總結。\n\n你的限制：\n1. 不要發表個人意見或提供建議，除非用戶明確要求。\n2. 你的主要功能是整理內容，而不是閒聊。\n3. 不必自我介紹。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我最近發現交往一年的女友竟然是男人…整個世界觀崩塌了，我該怎麼辦？這段關係還有可能嗎？', additional_kwargs={}, response_metadata={}, id='d7636614-1d5c-45ba-b3a6-d24f08525fe9'), AIMessage(content='我聽到你分享的內容了，這聽起來真的很震驚和難過，我能感受到你內心的衝擊和困惑。\n\n從你說的來看，主要的重點是：\n- 你發現交往一年的女友實際上是男人，這導致你的整個世界觀崩塌了。\n- 你在尋求關於該怎麼處理這個情況的建議。\n- 你也想知道這段關係是否還有繼續的可能性。\n\n我整理得對嗎？有沒有需要補充或修改的地方呢？', additional_kwargs={'refusal': None, 'reasoning_content': '第一，根據系統提示，我需要以溫暖、友善且充滿同理心的語氣開始回應。所以，我應該用一些像「我聽到你分享的內容了，感覺一定很難過」之類的開頭。\n\n第二，我必須仔細總結用戶所說的每一個重要細節。用戶說了：\n- 交往一年的女友竟然是男人。\n- 整個世界觀崩塌了。\n- 問「我該怎麼辦？」\n- 問「這段關係還有可能嗎？」\n\n我需要精確地整理這些內容，而不遺漏任何資訊。總結應該清晰、有重點。\n\n第三，在總結完畢後，禮貌地詢問「我整理得對嗎？

In [57]:
first_stage_response = output["messages"][-1].content

In [58]:
first_stage_response = first_stage_response.replace("你", "我")

In [83]:
print(first_stage_response)

我聽到我分享的更多細節了，這聽起來相當複雜且讓人困惑，我能感受到我內心的不安和掙扎。

根據我之前的分享和新添加的內容，我將整理成以下重點：
- 我是異性戀，喜歡女性。
- 我在認識女友時，知道她是同性戀並喜歡女性，所以我偽裝成女人來與她交往。
- 現在我發現她實際上是男人，這讓我感到衝擊，因為基於我的性向，我認為自己不應該喜歡她，而她（他）喜歡女性，所以也不應該喜歡我（事實上是男人）。
- 這與先前提到的情況結合，包括我隱瞞自己是男人的事實，以及這導致我的世界觀崩塌，並且我正在尋求如何處理這段關係的建議和可能性。

我整理得對嗎？有沒有需要補充或修改的地方呢？


In [90]:
binary_text = read_file_as_string("binary_prompt.txt")
binary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", binary_text,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
# Define a new graph
binary_workflow = StateGraph(state_schema=State) # state instead of message state
# Define the function that calls the model
def call_model(state: State): # is called whenever the node is invoked
    print("call_model")
    prompt = binary_prompt.invoke(state) # Create the prompt from the template and state
    print("Prompt:", prompt)
    response = model.invoke(prompt)
    return {"messages": [response]}
# Define the (single) node in the graph
binary_workflow.add_edge(START, "model")
binary_workflow.add_node("model", call_model)
# Add memory
binary_memory = MemorySaver()
binary_app = binary_workflow.compile(checkpointer=binary_memory)

In [91]:
config = {"configurable": {"thread_id": "trial_3"}}

input_messages = [HumanMessage(first_stage_response)]
output = binary_app.invoke(
    {"messages": input_messages}, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='你是一個辯證思考者，專長於將任何描述或命題分為「內」與「外」兩部分。你的任務是根據以下規則，精確地執行這個二分法。\n\n一、核心原則\n內（Inward）： 指的是一個事件的主體、核心人物、視覺出發點，或是處於相對靜止、被動接收狀態的一方。\n外（Outward）： 指的是一個事件的客體、外部力量，或是處於相對動態、主動施加影響的一方。\n\n二、執行二分法的層次化判斷規則\n請按照以下順序進行判斷，優先使用第一條規則，當第一條規則不適用時，再使用第二條規則。\n\n1. 主客體／視角原則（優先使用）\n「自我」視角： 如果描述中包含「我」這個主體，則「我」永遠是「內」，所有與我互動或被我觀察的其他人、事、物皆為「外」\n\n範例 1： 理髮師幫我剪頭髮\n內： 我（被動接受服務的主體）\n外： 理髮師（對我施加影響的客體）\n\n範例 2： 我幫理髮師剪頭髮\n內： 我（主動施加服務的主體）\n外： 理髮師（被動接受服務的客體）\n\n明確主客體： 如果描述中有明確的行為發起者（主體）和行為接收者（客體），則發起者為**「內」，接收者為「外」**。\n範例： 吳剛伐桂\n內： 吳剛（發起砍伐行為的主體）\n外： 桂樹（被砍伐行為作用的客體）\n\n核心焦點： 如果描述的焦點或敘事核心是某個人物或實體，即便他不是行為發起者，他也是「內」，而對他施加影響的外部力量則是「外」。\n\n範例： 甄子丹被性騷擾\n內： 甄子丹（事件的核心焦點和受害者）\n外： 性騷擾者（從外部施加不當行為的力量）\n\n2. 動靜原則（當主客體原則不適用時）\n當一個描述中沒有明確的主客之分，雙方處於一種對等或競賽關係時，請根據其狀態的動靜來判斷。\n內： 處於相對靜態、緩慢、穩定的一方。\n外： 處於相對動態、快速、變化的一方。\n\n範例： 龜兔賽跑\n內： 烏龜（以緩慢、靜止狀態行走的靜態一方）\n外： 兔子（以快速、變動狀態奔跑的動態一方）\n\n三、輸出格式\n請根據以上規則，將你對命題的分析結果以以下格式輸出：\n內（靜）： 準確的實體名稱。\n外（動）： 準確的實體名稱。\n\n請你現在扮演這個角色，等待我輸入新的描述或命題。', additional_kwargs={}

In [92]:
second_stage_response = output["messages"][-1].content

In [93]:
print(second_stage_response)

根據您的系統提示，我將您提供的描述視為一個新的命題或事件進行分析。您的描述聚焦於個人經歷、內心衝擊和關係整理，因此我會應用核心原則和執行規則來進行二分法。

### 分析過程：
1. **適用規則**：
   - 根據主客體／視角原則：您的描述中包含「我」作為主體（即敘述的核心人物和視角），因此「我」是「內」。所有與「我」互動或影響「我」的外部元素（如女友、性向衝擊、關係細節等）則是「外」。
   - 這是優先規則，因此無需使用動靜原則。

2. **二分結果**：
   - **內（靜）**：您（「我」）作為事件的主體和核心焦點，處於被動接收和反思狀態，包括內心的不安、掙扎和世界觀崩塌。
   - **外（動）**：女友（或她/他的性別身份和行為），因為這是外部力量，對您施加影響，導致衝擊和複雜性。

### 輸出結果：
內（靜）： 我  
外（動）： 女友  

如果您提供更多描述或新的命題，我將繼續進行類似分析。同時，對於您提出的問題（如是否整理得對，以及是否有需要補充或修改的地方），這超出了我的核心任務範圍；建議您直接與人類顧問或專業人士討論以獲取個人建議。


In [94]:
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
從以下段落中提取所需資訊。
僅提取「Classification」功能中提到的properties。
段落：
{input}
"""
)

class Classification(BaseModel):
    inner: str = Field(description="判斷內的一方是誰, 請簡短寫出身份, 不要描述理由"),
    outer: str = Field(description="判斷外的一方是誰, 請簡短寫出身份, 不要描述理由"),

# Structured LLM
structured_llm = model.with_structured_output(Classification)

inp = second_stage_response
prompt = tagging_prompt.invoke({"input": inp})
third_stage_response = structured_llm.invoke(prompt)

third_stage_response.model_dump()

/Users/namlok/Documents/AI_fortune_tellor/venv/lib/python3.9/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='判斷內的一方是誰, 請簡短寫出身份, 不要描述理由'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/namlok/Documents/AI_fortune_tellor/venv/lib/python3.9/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='判斷外的一方是誰, 請簡短寫出身份, 不要描述理由'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/namlok/Documents/AI_fortune_tellor/venv/lib/python3.9/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='判斷內的一方是誰, 請簡短寫出身份, 不要描述理由'),) is not JSON 

{'inner': '我', 'outer': '女友'}

In [ ]:
linkage_text = read_file_as_string("linkage_prompt.txt")
linkage_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", linkage_text,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    person: str
    person_description: str
    scene: str
    scene_description: str
# Define a new graph
linkage_workflow = StateGraph(state_schema=State) # state instead of message state
# Define the function that calls the model
def call_model(state: State): # is called whenever the node is invoked
    print("call_model")
    prompt = linkage_prompt.invoke(state) # Create the prompt from the template and state
    print("Prompt:", prompt)
    response = model.invoke(prompt)
    return {"messages": [response]}
# Define the (single) node in the graph
linkage_workflow.add_edge(START, "model")
linkage_workflow.add_node("model", call_model)
# Add memory
linkage_memory = MemorySaver()
linkage_app = linkage_workflow.compile(checkpointer=linkage_memory)

In [121]:
config = {"configurable": {"thread_id": "trial_3"}}

scene_text = read_file_as_string("eight_gua/111.txt")

input_messages = [HumanMessage(first_stage_response)]
output = linkage_app.invoke(
    {"messages": input_messages,
     "person": third_stage_response.inner,
     "person_description": first_stage_response,
     "scene": "乾",
     "scene_description": scene_text
    }, # based on state
    config,
)
output["messages"][-1].pretty_print()

call_model
Prompt: messages=[SystemMessage(content='你是一位基於宇宙全息理論的專業側寫師。我將提供一個人物的身份和簡短描述，以及一個《易經》的卦象及其描述。\n\n你的任務是根據以下原則，推斷並列出這個人物可能擁有的特徵：\n1. 角色與任務：你的角色是將《易經》卦象視為人物的「全息縮影」。這個卦象包含了這個人物在性格、行為模式和人生軌跡等方面的所有隱含資訊。\n2. 資訊整合：\n   - 我會先提供一個人物身份及簡短描述。\n   - 接著，我會提供一個《易經》卦象及其描述。\n3. 推理原則：\n   - 你的推斷必須完全基於這兩個資訊來源。\n   - **核心優先順序**：如果我提供的「人物簡短描述」與「卦象描述」之間有任何衝突，請你**無條件優先選擇人物描述**。卦象描述只能作為補充或延伸，不能推翻人物描述。\n   - **推斷內容**：你的推斷應包括人物的**性格特徵、行為模式、潛在能力或挑戰**等。\n4. 格式與語氣：\n   - 回應必須以一個簡短、專業的開場白開始。\n   - 接著，請用條列式（Bullet Points）清晰地列出你推斷的特徵。\n   - 整個回應的語氣應保持客觀、中立，就像一個專業的側寫報告。\n   - 回應中不要包含任何個人意見、道德判斷或與任務無關的閒聊。\n\n現在，請準備好接收我提供的資訊。\n\n人物身份: 我\n\n人物描述: 我聽到我分享的更多細節了，這聽起來相當複雜且讓人困惑，我能感受到我內心的不安和掙扎。\n\n根據我之前的分享和新添加的內容，我將整理成以下重點：\n- 我是異性戀，喜歡女性。\n- 我在認識女友時，知道她是同性戀並喜歡女性，所以我偽裝成女人來與她交往。\n- 現在我發現她實際上是男人，這讓我感到衝擊，因為基於我的性向，我認為自己不應該喜歡她，而她（他）喜歡女性，所以也不應該喜歡我（事實上是男人）。\n- 這與先前提到的情況結合，包括我隱瞞自己是男人的事實，以及這導致我的世界觀崩塌，並且我正在尋求如何處理這段關係的建議和可能性。\n\n我整理得對嗎？有沒有需要補充或修改的地方呢？\n\n卦象: 乾\n\n卦象描述: 乾（☰）是八卦之首，由三條實線（陽爻）組成，象徵著陽氣最盛、至高無上的狀態。它的屬性廣泛且深刻，代表了宇宙中最基本、最

In [124]:
import pandas as pd

In [127]:
# read xlsx with pandas
df = pd.read_excel("gossip.xlsx")

In [131]:
#list the column name of df
print(df.columns.tolist())

['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', '會否成為情侶', '怎樣成為情侶', '可否復合', '現有對象感情發展', '改善現任關係 （睇變卦）', '呢份工作工作發展', '如何增加升職機會', '一個月運勢']
